# Label side

In [ ]:
import numpy as np
from jesse import helpers, research

_, candles = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-06-18"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

candles.shape

In [2]:
np.save("data/btc_1m.npy", candles)

In [ ]:
import numpy as np

candles = np.load("data/btc_1m.npy")
candles.shape

In [ ]:
from scipy import stats
from custom_indicators.toolbox.bar.fusion.v0 import FusionBarContainerV0

bar_container = FusionBarContainerV0(max_bars=500000)
bar_container.update_with_candles(candles)
merged_bar = bar_container.get_fusion_bars()
close_arr = merged_bar[:, 2]
ret = np.log(close_arr[1:] / close_arr[:-1])
standard = (ret - ret.mean()) / ret.std()
kurtosis = stats.kurtosis(standard, axis=None, fisher=False, nan_policy="omit")
kurtosis

In [ ]:
from custom_indicators.utils.plot import plot_kde

L = 5

plot_kde(merged_bar[:, 2], lag=L)

In [ ]:
from hmmlearn.hmm import GMMHMM
import pandas as pd
import optuna

from jesse.helpers import timestamp_to_time


def objective(trial: optuna.Trial):
    mix = 3  ### GMM mix参数

    close_arr = merged_bar[:, 2]
    high_arr = merged_bar[:, 3][L:]
    low_arr = merged_bar[:, 4][L:]

    log_return = np.log(close_arr[1:] / close_arr[:-1])[L - 1 :]
    log_return_L = np.log(close_arr[L:] / close_arr[:-L])
    HL_diff = np.log(high_arr / low_arr)

    X = np.column_stack([HL_diff, log_return_L, log_return])

    datelist = np.asarray(
        [pd.Timestamp(timestamp_to_time(i)) for i in merged_bar[:, 0][L:]]
    )
    closeidx = merged_bar[:, 2][L:]

    assert len(datelist) == len(closeidx)
    assert len(datelist) == len(X)

    gmm = GMMHMM(
        n_components=2,
        n_mix=mix,
        covariance_type="diag",
        n_iter=1000,
        # weights_prior=2,
        means_weight=0.5,
        random_state=trial.suggest_int("random_state", 0, 1000),
    )
    gmm.fit(X)
    latent_states_sequence = gmm.predict(X)
    data = pd.DataFrame(
        {
            "datelist": datelist,
            "logreturn": log_return,
            "state": latent_states_sequence,
        }
    ).set_index("datelist")

    final_ret = 0
    for i in data["state"].unique():
        ret = data[data["state"] == i]["logreturn"].sum()
        final_ret += np.abs(ret)

    return final_ret


study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=20)

In [ ]:
study.best_params

In [ ]:
from hmmlearn.hmm import GMMHMM  # noqa
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd  # noqa

from jesse.helpers import timestamp_to_time  # noqa

mix = 3  ### GMM mix参数

close_arr = merged_bar[:, 2]
high_arr = merged_bar[:, 3][L:]
low_arr = merged_bar[:, 4][L:]

log_return = np.log(close_arr[1:] / close_arr[:-1])[L - 1 :]
log_return_L = np.log(close_arr[L:] / close_arr[:-L])
HL_diff = np.log(high_arr / low_arr)

X = np.column_stack([HL_diff, log_return_L, log_return])

datelist = np.asarray(
    [pd.Timestamp(timestamp_to_time(i)) for i in merged_bar[:, 0][L:]]
)
closeidx = merged_bar[:, 2][L:]

assert len(datelist) == len(closeidx)
assert len(datelist) == len(X)

gmm = GMMHMM(
    n_components=2,
    n_mix=mix,
    covariance_type="diag",
    n_iter=1000,
    # weights_prior=2,
    means_weight=0.5,
    random_state=study.best_params["random_state"],
)
gmm.fit(X)
latent_states_sequence = gmm.predict(X)

print(np.unique(latent_states_sequence, return_counts=True))

fig = go.Figure()
colors = px.colors.qualitative.Plotly

for i in range(gmm.n_components):
    state = latent_states_sequence == i
    fig.add_trace(
        go.Scatter(
            x=datelist[state],
            y=closeidx[state],
            mode="markers",
            name=f"latent state {i}",
            marker=dict(color=colors[i % len(colors)], size=4),
        )
    )

fig.update_layout(
    title="隐含状态序列",
    xaxis_title="时间",
    yaxis_title="收盘价",
    # width=1400,
    # height=600,
    showlegend=True,
)

fig.show()

In [ ]:
import matplotlib.pyplot as plt

data = pd.DataFrame(
    {
        "datelist": datelist,
        "logreturn": log_return,
        "state": latent_states_sequence,
    }
).set_index("datelist")

for i in data["state"].unique():
    ret = data[data["state"] == i]["logreturn"].sum()
    count = data[data["state"] == i].shape[0]
    print(f"state {i} ({count}) return: {ret:.6%}")

plt.figure(figsize=(20, 8))
for i in range(gmm.n_components):
    state = latent_states_sequence == i
    idx = np.append(0, state[1:])
    data["state %d_return" % i] = data.logreturn.multiply(idx, axis=0)
    plt.plot(np.exp(data["state %d_return" % i].cumsum()), label="latent_state %d" % i)
    plt.legend(loc="upper left")
    plt.grid(1)

In [12]:
assert len(merged_bar[L:]) == len(latent_states_sequence)

In [ ]:
# 保存merged bar, 去除最后一个bar，因为需要label前移一位
print(merged_bar[:-1].shape)
np.save("data/merged_bar.npy", merged_bar[:-1])

In [ ]:
# 上涨下跌统一打标的情况

label = (latent_states_sequence == 1).astype(int)
print(np.unique(label, return_counts=True))
np.save("data/side_label.npy", label)

In [ ]:
# 上涨下跌分别打标的情况

# label_long = (latent_states_sequence == 1).astype(int)
# print(np.unique(label_long, return_counts=True))
# np.save("data/side_label_long.npy", label_long)

# label_short = (latent_states_sequence == 0).astype(int)
# print(np.unique(label_short, return_counts=True))
# np.save("data/side_label_short.npy", label_short)

# Label meta

In [1]:
import numpy as np
import pandas as pd
from strategies.BinanceBtcEntropyBarV1.config import SIDE
from strategies.BinanceBtcEntropyBarV1.config import get_side_model

side_model = get_side_model(False)

df_features = pd.read_parquet("data/features.parquet")

side_res = side_model.predict(df_features[SIDE])

side_pred_label = np.where(side_res > 0.5, 1, -1)
print(side_pred_label.shape)

print(np.unique(side_pred_label, return_counts=True))

side_label = np.load("data/side_label.npy")
side_label = np.where(side_label == 1, 1, -1)

len_gap = len(side_label) - len(side_pred_label)
side_label = side_label[len_gap:]

print(np.unique(side_label, return_counts=True))

(34487,)
(array([-1,  1]), array([15541, 18946]))
(array([-1,  1]), array([15551, 18936]))


In [2]:
meta_label = (side_pred_label == side_label).astype(int)
print(meta_label.shape)

merged_bar = np.load("data/merged_bar.npy")
log_ret = np.log(merged_bar[1:, 2] / merged_bar[:-1, 2])
len_gap = len(log_ret) - len(side_pred_label)
log_ret = log_ret[len_gap:]
print(log_ret.shape)

# start_idx = 0
# cumsum_ret = 0
# for idx, (i, r) in enumerate(zip(side_pred_label[:-1], log_ret[:-1])):
#     if i == 1:
#         if idx == 0:
#             # 开始持仓
#             start_idx = idx
#         elif side_pred_label[idx - 1] == -1:
#             # 反向持仓，先结算收益
#             if cumsum_ret > 0:
#                 meta_label[start_idx:idx] = 1
#             cumsum_ret = 0
#             start_idx = idx
#         else:
#             # 继续持仓
#             cumsum_ret += r
#     elif i == -1:
#         if idx == 0:
#             # 开始持仓
#             start_idx = idx
#         elif side_pred_label[idx - 1] == 1:
#             # 反向持仓，先结算收益
#             if cumsum_ret < 0:
#                 meta_label[start_idx:idx] = 1
#             cumsum_ret = 0
#             start_idx = idx
#         else:
#             # 继续持仓
#             cumsum_ret += r
#     else:
#         raise ValueError(f"side_pred_label[{idx}] = {i} is not valid")


print(np.unique(meta_label, return_counts=True))

# meta label复核：对于所有meta label = 1的情况，结合side_pred_label看是否真的盈利
TRADE_FEE = 0.05 / 100

start_idx = 0
cumsum_ret = 0
for idx, (meta, side, ret) in enumerate(zip(meta_label, side_pred_label, log_ret)):
    if meta == 1:
        if idx > 0 and meta_label[idx - 1] == 0:
            # 开始持仓
            start_idx = idx
            cumsum_ret -= TRADE_FEE
        else:
            # 继续持仓
            cumsum_ret += ret * side
    elif meta == 0:
        if idx > 0 and meta_label[idx - 1] == 1:
            # 结束持仓
            cumsum_ret += ret * side - TRADE_FEE
            end_idx = idx
            if cumsum_ret < 0:
                # 如果收益为负，则认为判断错误
                assert start_idx < end_idx, "start_idx must be less than end_idx"
                meta_label[start_idx:end_idx] = 0
            # 重置收益
            cumsum_ret = 0
            start_idx = 0
        else:
            continue

print(np.unique(meta_label, return_counts=True))

(34487,)
(34487,)
(array([0, 1]), array([  266, 34221]))
(array([0, 1]), array([  311, 34176]))


In [9]:
np.save("data/label_meta.npy", meta_label)